# CS498 Project, Team 31, Phase 1
## Sentiment Analysis of Social Media as a predictor for Bitcoin price volatility

Phase 1 of the project will be to conduct a data exploration of the cryptocurrency historical datasets to ascertain if a correlation can be drawn between Bitcoin price fluctuations and the other major cryptocurrencies. The comparison will be done using the top ten cryptocurrencies by market share. The top ten were chosen based by market cap as of Dec. 7, 2017, according to CoinMarketCap.com. The historical data will be restricted to the last 90 days of the year 2017.

1. Bitcoin (BTC): \$304.52 billion
2. Ethereum (ETH): \$41.55 billion
3. Bitcoin Cash (BCH): \$22.02 billion
4. IOTA (IOT): \$11.59 billion
5. Ripple (XRP): \$8.68 billion
6. Dash (DASH): \$5.38 billion
7. Litecoin (LTC): \$5.32 billion
8. Monero (XMR): \$4.33 billion
9. Bitcoin Gold (BTG): \$4.22 billion
10. Cardano (ADA): \$2.78 billion
11. Ethereum Classic (ETC): \$2.62 billion
12. Stellar Lumens (XLM): \$2.53 billion
13. NEO (NEO): \$2.23 billion

#### Because some coins didnt start trading until late 2017, the following substitutions were made:
* Ethereum classic was exchanged for IOTA
* Stellar Lumens was exchanged for Cardano
* Neo was exchanged for Bitcoin Gold


The datasets used were downloaded from the [CryptoCompare public API](https://www.cryptocompare.com/api/#introduction), we can use the following Python wrapper available on [GitHub: cryCompare](https://github.com/stefs304/cryCompare) to simplfy the API usage.


In [82]:
# Standard Data Science libraires to be used
%autoreload 2

import numpy as np
import pandas as pd
import operator
import seaborn as sns
color = sns.color_palette()
import matplotlib.pyplot as plt
import sys

# This path will need to be modified based on where to save the crycompare library
sys.path.append("C:/Users/Michael/notebooks/CS498/crycompare/")
from crycompare import history as h 
from crycompare import price as p 

%matplotlib inline
%matplotlib inline

In [129]:
# Fetch a coin list from CryptoCompare and create a sub list with only coins of interest.
top10_coins = ['BTC', 'ETH', 'BCH', 'ETC','XRP', 'DASH', 'LTC', 'XMR', 'NEO', 'XLM'] # The top ten coins for 2017
# It was not nessecary to fetch the coin list since it is simpler to create a list of the coins we want to fetch.
# coinList = p.coinList()
# top10_coinList = dict((k, coinList[k]) for k in top10_coins if k in coinList)
coins = sorted(top10_coins)
print(coins)

['BCH', 'BTC', 'DASH', 'ETC', 'ETH', 'LTC', 'NEO', 'XLM', 'XMR', 'XRP']


In [126]:
# With the histoDay() function we can fetch the historical data (OHLC prices and volumes) open-high-low-close chart.
df_dict = {}
for coin in coins:
    histo = h.histoDay(coin, 'USD', limit=89, toTs=1514678400, allData=False)
    if histo:
        df_histo = pd.DataFrame(histo)
        df_histo['time'] = pd.to_datetime(df_histo['time'],unit='s')
        df_histo.index = df_histo['time']
        del df_histo['time']
        del df_histo['volumefrom']
        del df_histo['volumeto']
        
        df_dict[coin] = df_histo


In [127]:
# We store all info in a dataframe with 2-level columns: the first level contains the coin names, the second one, the OHLC prices.
top10_coin_history = pd.concat(df_dict.values(), axis=1, keys=df_dict.keys())


In [128]:
top10_coin_history.head()

BCH                              BTC                    \
             close    high     low    open    close     high      low   
time                                                                    
2017-10-03  401.41  420.69  393.03  419.68  4314.18  4435.99  4230.73   
2017-10-04  355.75  403.22  351.30  401.41  4218.66  4355.29  4183.57   
2017-10-05  354.40  368.14  343.72  355.75  4321.44  4365.76  4143.35   
2017-10-06  363.22  374.54  352.91  354.40  4371.94  4422.15  4296.93   
2017-10-07  359.81  368.17  354.21  363.22  4435.81  4472.92  4319.48   

                       DASH                            ETC                \
               open   close    high     low    open  close   high    low   
time                                                                       
2017-10-03  4401.32  294.50  310.08  288.66  307.98  12.29  12.54  11.78   
2017-10-04  4314.18  303.32  317.21  294.17  294.50  11.79  12.39  11.78   
2017-10-05  4218.66  305.35  307.80  293.16  303.32  11.91  12.08  11.51   
2017-10-06  4321.44  305.95  312.71  303.10  305.35  12.33  12.41  11.90   
2017-10-07  4371.94  311.52  315.15  301.90  305.95  12.37  12.55  12.04   

                      ETH                            LTC                       \
             open   close    high     low    open  close   high    low   open   
time                                                                            
2017-10-03  12.43  291.81  299.92  286.86  296.81  52.15  53.54  51.34  53.29   
2017-10-04  12.29  291.68  295.57  288.56  291.81  51.22  52.98  50.53  52.15   
2017-10-05  11.79  294.99  297.47  286.73  291.68  51.64  51.87  50.17  51.22   
2017-10-06  11.91  308.33  309.00  293.75  294.99  52.08  52.75  51.34  51.64   
2017-10-07  12.33  311.26  314.12  304.63  308.33  52.57  52.90  51.61  52.08   

              NEO                           XLM                             \
            close   high    low   open    close     high      low     open   
time                                                                         
2017-10-03  33.57  36.77  31.88  36.61  0.01241  0.01300  0.01126  0.01300   
2017-10-04  31.17  35.29  30.10  33.57  0.01181  0.01284  0.01150  0.01241   
2017-10-05  32.28  33.63  28.83  31.17  0.01449  0.01515  0.01158  0.01181   
2017-10-06  33.49  34.47  32.08  32.28  0.01838  0.01927  0.01449  0.01449   
2017-10-07  35.88  36.27  33.16  33.49  0.02077  0.02470  0.01838  0.01838   

              XMR                          XRP                          
            close   high    low   open   close    high     low    open  
time                                                                    
2017-10-03  92.99  94.74  87.13  91.39  0.2033  0.2062  0.1980  0.2031  
2017-10-04  90.24  93.62  88.51  92.99  0.2135  0.2170  0.2012  0.2033  
2017-10-05  91.88  92.45  87.78  90.24  0.2380  0.2442  0.2134  0.2135  
2017-10-06  91.05  92.82  89.76  91.88  0.2334  0.2444  0.2332  0.2380  
2017-10-07  91.61  92.41  89.25  91.05  0.2397  0.2411  0.2324  0.2334

In [124]:
top10_coin_history.tail()

BCH                                  BTC                      \
              close     high      low     open     close      high       low   
time                                                                           
2017-12-27  2710.64  2924.47  2568.85  2921.89  15416.64  16514.59  14534.66   
2017-12-28  2484.96  2727.03  2291.94  2710.64  14398.70  15505.51  13466.07   
2017-12-29  2619.32  2884.96  2311.24  2486.54  14392.57  15109.81  13951.08   
2017-12-30  2209.96  2634.36  2103.91  2622.42  12531.52  14461.46  11962.09   
2017-12-31  2371.83  2485.77  2165.12  2210.18  13850.40  14241.82  12359.43   

                         BTG                             DASH           \
                open   close    high     low    open    close     high   
time                                                                     
2017-12-27  15757.02  270.49  294.59  260.43  274.36  1133.36  1210.54   
2017-12-28  15416.34  246.85  271.21  225.66  270.50  1103.06  1138.81   
2017-12-29  14398.45  252.99  272.61  240.12  246.85  1089.31  1144.02   
2017-12-30  14392.14  216.91  255.25  207.91  253.00   943.14  1095.32   
2017-12-31  12532.38  240.89  246.18  212.58  216.91  1013.89  1054.13   

                                ETC                          ETH          \
                low     open  close   high    low   open   close    high   
time                                                                       
2017-12-27  1103.32  1181.28  28.78  30.74  27.59  29.99  739.94  766.53   
2017-12-28   992.24  1133.50  26.89  28.95  25.18  28.78  716.69  741.46   
2017-12-29  1023.52  1103.06  27.38  28.63  25.96  26.89  739.60  755.20   
2017-12-30   875.90  1089.16  23.84  27.37  22.77  27.35  692.99  741.67   
2017-12-31   924.33   943.10  26.05  26.61  23.62  23.84  741.13  746.62   

                               LTC                             XLM          \
               low    open   close    high     low    open   close    high   
time                                                                         
2017-12-27  706.67  753.44  264.07  285.32  252.87  279.47  0.2270  0.2300   
2017-12-28  671.23  739.93  249.86  265.65  224.07  264.07  0.2174  0.2270   
2017-12-29  710.20  716.69  243.13  258.36  236.39  249.84  0.3170  0.3300   
2017-12-30  663.88  739.60  212.07  243.42  202.02  243.13  0.3090  0.3700   
2017-12-31  686.44  692.99  226.52  235.40  207.95  212.07  0.3540  0.3586   

                               XMR                           XRP              \
               low    open   close    high     low    open close  high   low   
time                                                                           
2017-12-27  0.2110  0.2221  375.87  409.15  342.36  357.33  1.21  1.28  1.07   
2017-12-28  0.1922  0.2270  349.95  376.59  325.25  375.87  1.26  1.31  1.10   
2017-12-29  0.2155  0.2174  349.19  367.77  335.96  349.95  1.93  1.97  1.25   
2017-12-30  0.2590  0.3160  304.06  352.37  288.08  349.22  1.86  2.44  1.77   
2017-12-31  0.2953  0.3043  331.83  340.84  300.76  304.06  1.98  2.09  1.61   

                  
            open  
time              
2017-12-27  1.07  
2017-12-28  1.21  
2017-12-29  1.26  
2017-12-30  1.93  
2017-12-31  1.86